# Ant Colony Algorithm for Set Cover Problem

## Introduction
This report presents an implementation of the Ant Colony Algorithm to solve the Set Cover Problem. 

## Algorithm Overview
The implemented algorithm follows these main steps:

1. **File Parsing:** The input file is parsed to extract information about the subsets, their costs, and the elements they cover. The subsets are represented using `bitarray` data structures, which provide fast and efficient operations for set manipulation.

2. **Ant Initialization:** Each ant is initialized with a random subset from the available collection. The ant's costs are set to zero, and an empty list of selected subsets is created.

3. **Ant Exploration:** Each ant explores the solution space by iteratively selecting subsets until all elements are covered. The selection process is based on the fitness of each subset, calculated using a combination of pheromone levels and the number of uncovered elements.

4. **Pheromone Update:** After all ants have completed their exploration, the pheromone levels are updated based on the sets selected by each ant. Pheromones are deposited on each subset proportional to the inverse of the ant's costs. This update reinforces the selection of subsets that contribute to better solutions.

5. **Evaporation of Pheromones:** To prevent the algorithm from getting stuck in suboptimal solutions, a pheromone evaporation process is applied. The pheromone levels for each subset are reduced based on an evaporation rate. This ensures exploration of a wider solution space and promotes diversity among ant solutions.


## Benefits of Pheromone Placement and Bitarray Usage
- **Pheromone Placement:** By placing pheromones on each subset, the algorithm encourages ants to select subsets that have been previously successful in finding good solutions. This exploitation of past knowledge guides the search towards potentially better solutions and accelerates convergence.

- **Bitarray Usage:** The adoption of `bitarray` data structures offers significant advantages in terms of speed and efficient set operations. Bitarrays allow for fast manipulation of subsets, such as element-wise logical operations, counting the number of uncovered elements, and merging subsets with minimal computational overhead.

- **Effect of Beta Parameter:** The `beta` parameter plays a role in balancing the exploration and exploitation aspects of the algorithm. A higher value of `beta` biases the selection process towards subsets with a larger number of uncovered elements, leading to a more greedy approach. This can result in faster convergence to a solution, especially when the optimal solution has subsets with a higher coverage. However, a very high value of `beta` may lead to premature convergence and suboptimal solutions. Hence, careful tuning of the `beta` parameter is necessary to strike a balance between exploration and exploitation while considering the characteristics of the problem at hand.

In conclusion, the implementation of the Ant Colony Algorithm for the Set Cover Problem demonstrates the effective utilization of pheromones on subsets and the use of `bitarray` data structures to enhance the algorithm's efficiency. By leveraging these techniques, the algorithm can efficiently explore the solution space and converge to optimal or near-optimal solutions.


In [13]:
from bitarray import bitarray
import random


In [1]:
def openFile(file):
    # Read the file and extract necessary values
    file = open(file).read()
    nums = list(map(int, file.split()))
    N1, N2 = nums[0], nums[1]
    nums = nums[2:]
    costs = [0 for _ in range(N2)]
    subSets = [[False for _ in range(N1)] for _ in range(N2)]

    # Extract costs for each subset
    for i in range(N2):
        costs[i] = nums[i]
    nums = nums[N2:]
    index = 0

    # Extract subsets and their elements
    for i in range(N1):
        count = nums[index]
        index += 1
        for _ in range(count):
            subSets[nums[index]-1][i] = True
            index += 1

    # Return subsets, costs, and indices as tuples
    return [(bitarray(subSets[i]), costs[i], i) for i in range(N2)]

In [2]:
class Ant:
    def __init__(self, subsets) -> None:
        # Initialize an ant with zero costs, an empty list of sets, and an empty bitarray for collected elements
        self.costs = 0
        self.sets = []
        self.collected = bitarray([False for _ in range(1, len(list(subsets[0][0]))+1)])

        # Choose a random subset and add it to the ant's sets
        i = random.randint(0, len(subsets)-1)
        self.addSet(subsets[i])

    def addSet(self, set):
        # Add a subset to the ant's sets and update collected elements and costs
        self.sets.append(set[2])
        self.collected |= set[0]
        self.costs += set[1]

    def putPheromone(self, pheromones):
        # Update the pheromone levels based on the ant's sets and costs
        for i in self.sets:
            pheromones[i] += 1/self.costs
        return len(self.sets)/self.costs

In [3]:
def calculateFitness(ant: Ant, set, alpha, beta, pheromones):
    # Calculate the fitness of a set for a given ant
    if(set[2] in ant.sets):
        return 0
    addedCount = (~ant.collected & set[0]).count()
    return (pheromones[set[2]]**alpha) * (addedCount/set[1])**beta

In [11]:
def ant_colony(file, antNumber, alpha, beta, evaporation_rate, iterationNumber, target) -> Ant:
    # Read subsets from the file
    subSets = openFile(file)

    # Initialize pheromone levels for subsets
    pheromones = [0.1 for _ in range(len(subSets))]

    # Run the ant colony algorithm for the specified number of iterations
    for _ in range(iterationNumber):
        # Create a list of ants
        ants = [Ant(subSets) for _ in range(antNumber)]

        # Iterate through each ant
        for ant in ants:
            ant: Ant

            # Continue until all elements are collected by the ant
            while not ant.collected.all():
                # Choose the next set based on fitness and add it to the ant's sets
                nextSet = random.choices(subSets, weights=[calculateFitness(ant, set, alpha, beta, pheromones) for set in subSets])[0]
                ant.addSet(nextSet)

            # Check if the ant's cost is less than or equal to the target
            if ant.costs <= target:
                return ant

        # Update the pheromone levels of each ant
        for ant in ants:
            ant: Ant
            ant.putPheromone(pheromones)

        # Update pheromone levels for each subset based on evaporation rate
        for i in range(len(subSets)):
            pheromones[i] *= (1 - evaporation_rate)


In [15]:
ant_colony('./scp41.txt',100,1,5,0.4,100,500).costs

499

In [18]:
ant_colony('./scp52.txt',100,1,5,0.4,100,360).costs

359

In [16]:
ant_colony('./scp62.txt',100,1,5,0.4,100,200).costs

194